In [1]:
from konlpy.tag import Okt
from collections import Counter
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
data = pd.read_csv('서비스 카페 블로그 크롤링(721개).csv',encoding='utf-8')

In [3]:
review = data['0'] # csv 파일에서 리뷰만 추출해옴

In [4]:
review

0      본격적인 여름인건지 엄청엄청 더워지기 시작했어요 .오늘은 낮 최고 기온이 35 이상...
1      모시러 운전동행서비스#시간제수행기사 #모시러 #운전동행안녕하세요 민사임당입니다. 운...
2      ㅌㅌㅌㅌㅌㅌㅌㅌㅊㅊㅊㅊㅊㅊㅊ 글씨체 테스트 ㅋㅋㅋ 오늘은 글씨체를 바꿔볼까요~? ㅎ...
3      싱그러운 5월의 초록이 아름다운 양재천봄엔 벚꽃, 여름엔 초록, 가을엔 노란 은행잎...
4      부천역 예쁜 카페 [블랙소울]노을진 하늘이 예쁘던 어느 날,계속 비가 오다가 오후에...
                             ...                        
716    족욕 카페에 가보신 적 있으신가요? 얼마 전에 여행 온 친구랑 같이 제주도 애월 카...
717    안녕하세요~  다들 크리스마스 잘 보내셨나요 ~? 오늘은 여러분들께 제 집앞쪽에있는...
718    벌써 세 번째 방문!위치는 조금 애매하지만..조용하니 분위기 너무 좋은 망포동 카페...
719    오늘 포스팅할 곳은 양재천에 위치한 핫한 카페입니다.주말에는 사람이 많을꺼 같아화요...
720    요즘 카페에서 공부하는 카공족 들이 늘어나면서 독서실과 카페를 결합한 스터디카페들이...
Name: 0, Length: 721, dtype: object

In [5]:
okt = Okt()
review_word = [] # 단어를 담을 리스트
review_value = [] # 단어가 나오는 횟수를 담을 리스트
for i in tqdm(range(review.size)):
    count = Counter(okt.nouns(review[i])) # 리뷰 0번, 1번...의 명사만 추출하기
    for key,value in count.items(): # count가 리스트 형태로 되어있어서 key, value를 추출
        if(value>=3): # 리뷰에서 3번 이상 나온 것만 담아주기
            review_word.append(key)
            review_value.append(value)

In [6]:
review_final = pd.DataFrame([review_word,review_value]) # 단어, 단어가 나온 횟수를 데이터프레임으로 만든다.

review_final # 옆으로 길게 나옴

,0,1,2,3,4,5,6,7,8,9,...,15091,15092,15093,15094,15095,15096,15097,15098,15099,15100
0,이,카페,그,맛,쪽,서면,때,커피,스가모,인,...,곳,이벤트,좌석,내용,실내,수,추가,구매,시,제공
1,5,22,3,3,4,16,4,17,4,4,...,5,6,5,3,3,5,5,4,4,5


In [7]:
review_final = review_final.T # 옆으로 길게 나와있는 데이터프레임을 행과 열을 바꾸어 세워준다

In [8]:
review_final

,0,1
0,이,5
1,카페,22
2,그,3
3,맛,3
4,쪽,4
...,...,...
15096,수,5
15097,추가,5
15098,구매,4
15099,시,4


In [9]:
for i in tqdm(range(len(review_final))): # 중복되는 단어를 찾는 시스템
    for j in range(i-1):
        if review_final[0][i] == review_final[0][j]: # 중복되는 단어가 나오면
            review_final[1][j] +=review_final[1][i] # 기존에 있는 숫자에 더해준다. 
            review_final[1][i] = 0;
            break
            # 1번 리뷰에서 카페가 3번, 2번 리뷰에서 카페가 4번 나오면 1번에다가 4를 더해주고 2번을 0으로 만들어버린다.
            # 처음에 있는 단어는 숫자가 계속 쌓이고 같은 단어의 나머지는 0으로 바뀐다

In [10]:
review_copy = review_final.copy() # 혹시라도 원본이 바뀔까봐 카피를 해놨음

In [11]:
zero = review_copy[review_copy[1] == 0].index # for문을 통해서 0값이 엄청 나왔을 것이고, 이것들을 제거해줘야함.

review_copy.drop(zero,inplace=True) # 0인 값들을 제거하는 부분

In [12]:
review_copy.sort_values(by=1,ascending=False) # 0을 지워서 가장 아래쪽에 있는 숫자가 3이다.(리뷰 한개에서만 쓴 글자다)

,0,1
1,카페,5913
7,커피,1876
10,곳,1656
95,것,1545
81,수,1450
...,...,...
7120,엔딩,3
7088,퓨전,3
7045,점도,3
7009,인형,3


In [13]:
review_final.sort_values(by=1,ascending=False) # 원본은 0을 안 지워서 아직 0이 그대로 뜬다. 

,0,1
1,카페,5913
7,커피,1876
10,곳,1656
95,것,1545
81,수,1450
...,...,...
6012,높이,0
6013,친구,0
6015,카페,0
6016,것,0


In [14]:
zero = review_final[review_final[1] == 0].index 

review_final.drop(zero,inplace=True) 
# 카피에서 이상이 없는거 확인했으니 원본도 0 값들을 지운다.

In [15]:
review_final.columns = ['단어 데이터','단어 횟수'] # 작업이 끝났으니 column 이름을 바꿔준다 (가독성)

In [16]:
review_final.sort_values(by='단어 횟수',ascending=False).head(10) # 가장 많이 나온 단어 10개만 뽑아봤다.

,단어 데이터,단어 횟수
1,카페,5913
7,커피,1876
10,곳,1656
95,것,1545
81,수,1450
14,분위기,1114
117,층,1076
3,맛,1007
22,주문,905
16,사진,806


In [17]:
review_final.sort_values(by='단어 횟수',ascending=False) # 원본도 이상이 없다는 것을 확인.

,단어 데이터,단어 횟수
1,카페,5913
7,커피,1876
10,곳,1656
95,것,1545
81,수,1450
...,...,...
7120,엔딩,3
7088,퓨전,3
7045,점도,3
7009,인형,3


In [18]:
review_copy.reset_index(drop=True, inplace=True) # 숫자가 어지럽게 되어있어서 다시 0부터 리셋시켰다.

review_copy

,0,1
0,이,428
1,카페,5913
2,그,88
3,맛,1007
4,쪽,110
...,...,...
3036,마우스,3
3037,항상,3
3038,망포동,6
3039,그루,5


In [19]:
review_copy = review_copy.sort_values(by=1,ascending=False) # 정렬된 값을 원본에 삽입

In [20]:
review_copy.reset_index(drop=True, inplace=True) # 원본에 삽입 후 인덱스 리셋하기

review_copy

,0,1
0,카페,5913
1,커피,1876
2,곳,1656
3,것,1545
4,수,1450
...,...,...
3036,엔딩,3
3037,퓨전,3
3038,점도,3
3039,인형,3


In [21]:
review_final = review_final.sort_values(by='단어 횟수',ascending=False)

review_final.reset_index(drop=True, inplace=True) # 원본에 삽입 후 인덱스 리셋하기

review_final

,단어 데이터,단어 횟수
0,카페,5913
1,커피,1876
2,곳,1656
3,것,1545
4,수,1450
...,...,...
3036,엔딩,3
3037,퓨전,3
3038,점도,3
3039,인형,3


In [22]:
review_final.to_csv('서비스 카페 단어사전 전처리.csv', index=False)